In [1]:
import dynet as dy
import numpy as np

In [2]:
## ==== Create a new computation graph
# (it is a singleton, we have one at each stage.
# renew_cg() clears the current one and starts anew)
dy.renew_cg()

## ==== Creating Expressions from user input / constants.
x = dy.scalarInput(1)

v = dy.vecInput(3)
v.set([1,2,3])

z = dy.matInput(2, 3)

# for example:
z1 = dy.matInput(2, 2)
z1.set([1,2,3,4])

## ==== We can take the value of an expression. 
# For complex expressions, this will run forward propagation.
print z.value()    
print z.npvalue()      # as numpy array
print v.vec_value()    # as vector, if vector
print x.scalar_value() # as scalar, if scalar
print x.value()        # choose the correct one


[[ 0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]]
[1.0, 2.0, 3.0]
1.0
1.0


In [3]:
## ==== Parameters
# Parameters are things we tune during training.
# Usually a matrix or a vector.

# First we create a model and add the parameters to it.
m = dy.Model() 
pW = m.add_parameters((8,8)) # an 8x8 matrix
pb = m.add_parameters(8)

# then we create an Expression out of the model's parameters
W = dy.parameter(pW)
b = dy.parameter(pb)

## ===== Lookup parameters
# Similar to parameters, but are representing a "lookup table"
# that maps numbers to vectors.
# These are used for embedding matrices.
# for example, this will have VOCAB_SIZE rows, each of DIM dimensions.
VOCAB_SIZE=10
DIM=5
DIMC=3
lp  = m.add_lookup_parameters((VOCAB_SIZE, DIM))
clp  = m.add_lookup_parameters((VOCAB_SIZE, DIMC))
attn = m.add_parameters((DIMC,DIM))

# lookup parameters can be initialized from an existing array, i.e:
# m["lookup"].init_from_array(wv)


In [4]:
lp.shape()

(10, 5)

In [5]:
e = lp[1]
A = dy.parameter(attn)

In [6]:
A.npvalue().shape

(3, 5)

In [7]:
e.npvalue().shape

(5,)

In [8]:
dy.transpose(e).npvalue().shape

(1, 5)

In [9]:
(A*e).npvalue().shape

(3,)

In [10]:
(dy.transpose(A*e)*clp[1]).npvalue()

array([-0.13906434])

In [11]:
(dy.transpose(clp[1])*(A*e)).npvalue()

array([-0.13906434])

In [12]:
attn_w = [dy.transpose(clp[i])*(A*e) for i in xrange(5)]
attn_w

[exprssion 21/1,
 exprssion 25/1,
 exprssion 29/1,
 exprssion 33/1,
 exprssion 37/1]

In [13]:
dy.softmax(dy.concatenate(attn_w)).npvalue()

array([ 0.08688716,  0.16628774,  0.09851928,  0.52598399,  0.12232184])

In [14]:
attn_w = dy.softmax(dy.concatenate(attn_w))
attn_w.npvalue()

array([ 0.08688716,  0.16628774,  0.09851928,  0.52598399,  0.12232184])

In [15]:
(clp[0]*attn_w[0]).npvalue()

array([ 0.0369454 , -0.01742968,  0.0734873 ])

In [19]:
attn_w

exprssion 41/1